In [ ]:
from Model import Generator, Discriminator
import os
import numpy as np
import math
import time
import random
from collections import defaultdict
from pathlib import Path
from Model import Generator

import torchvision.transforms as transforms
from torchvision.utils import save_image
from torch.utils.data import DataLoader, random_split, Dataset
from torchvision import datasets
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch

import matplotlib.pyplot as plt

In [ ]:
transform = transforms.Compose([
  transforms.ToTensor()
])

class rain(Dataset):

    def __init__(self, input_dir, output_dir, transform=transform):
        self.input_dir = Path(input_dir)
        self.output_dir = Path(output_dir)
        self.input = np.load(self.input_dir)
        self.output = output = np.load(self.output_dir)
        self.transform = transform

    def __len__(self):
        return len(self.input)

    def __getitem__(self, idx):   #idx就是来选取第几个的那个index
        input = self.input[idx]
        output = self.output[idx] 

        if self.transform:
            input = self.transform(input)  #torch的tensor和np实际上是反过来的，我们需要改动一下
            output = self.transform(output)

        return input, output


in_dir = 'paddedinput_30mins_4ch.npy'
out_dir = 'paddedoutput_30mins_4ch.npy'
val_percent: float = 0.2 #Use 20% as validation dataset
batch_size = 16

dataset = rain(in_dir, out_dir, transform)     #创造一个dataset，用我上面自定义的dataset class

n_val = int(len(dataset) * val_percent)   #这几行就是定义多少个training 多少个validation
n_train = len(dataset) - n_val
train_set, val_set = random_split(dataset, [n_train, n_val], generator=torch.Generator().manual_seed(42))


#Dataloader了该 Dataloader的作用就是打包Batch
loader_args = dict(batch_size=batch_size, num_workers=4, pin_memory=True) #为下面的两行做准备
#train_loader = DataLoader(train_set, shuffle=True, **loader_args)   #dataloader来load这个dataset，分为train和validation
val_loader = DataLoader(val_set, shuffle=False, drop_last=True, **loader_args)

####################################


device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

generator = Generator().to(device)
generator.load_state_dict(torch.load('generator.pth'))
generator.eval()

unet =  Generator().to(device)
unet.load_state_dict(torch.load('checkpoint.pth'))
unet.eval()


def depad(pred, from_shape_w=192, to_shape_w=180, from_shape_h=128, to_shape_h=120):
    padding_w = int( (from_shape_w - to_shape_w) / 2)
    padding_h = int( (from_shape_h - to_shape_h) / 2)
    return pred[padding_w:padding_w+to_shape_w, padding_h:padding_h+to_shape_h].copy()



#This function picks up the nth batch from the validation dataset (val_loader)
def select_batch(batch_num):
    if batch_num >= len(val_loader): return -1    #if we dont have that much batch in val dataset, we should return error

    i = 0
    for data in iter(val_loader):
        input, output = data
        #print(x.size())
        if i >= batch_num: break    #define which batch (with size 16 in each) to see
        i += 1
    return input, output


def make_pred(input, output, model):    #This not only make a prediction, but also produce numpy
    input = input.to(device).float()
    output = output.to(device).float()

    if model == 'unet':
        pred = unet(input)
    else:
        pred = generator(input)

    pred = pred.data.cpu().numpy()
    input = input.data.cpu().numpy()
    output = output.data.cpu().numpy()

    return input, output, pred


In [ ]:
# Plot the rainfall
# Takes a batch of 16 as prediction/ground truth, select which frame from the batch to be plotted
def plot(prediction, ground, frame):

    fig = plt.figure(figsize=(9,6))

    fig.add_subplot(121)
    pred_image = depad(prediction[frame,0,:,:])
    plt.title("Prediction", fontsize = 30)
    plt.imshow(pred_image, cmap = 'jet', vmax = 10, vmin = 0)

    fig.add_subplot(122)
    
    output_image = depad(ground[frame,0,:,:])
    plt.title("Ground Truth", fontsize = 30)
    plt.imshow(output_image, cmap = 'jet', vmax = 10, vmin = 0)

    fig.tight_layout()

input, output = select_batch(batch_num = 0)   #Define specific batch to use
input, output, pred = make_pred(input, output, 'gan')
plot(pred, output, frame = 4)  #Define specific frame to see

In [ ]:
# Print the radially averaged PSD
import pysteps

frame = 8
input, output = select_batch(batch_num = 0)   #Define specific batch to use
input_np, output_np, pred_gan = make_pred(input, output, 'gan')
input_np, output_np, pred_unet = make_pred(input, output, 'unet')

output_depad = depad(output_np[frame,0,:,:])
pred_gan_depad = depad(pred_gan[frame,0,:,:])
pred_unet_depad = depad(pred_unet[frame,0,:,:])

plt.figure(figsize=(6, 9))

ax = plt.subplot(111)
R_obs_spectrum, fft_freq = pysteps.utils.rapsd(output_depad, np.fft, d=1.0, return_freq=True)
wavelength_ticks = [512,256,128,64,32,16,8,4,2]   #### NEED CLARIFICATION
pysteps.plt.plot_spectrum1d(fft_freq, R_obs_spectrum, x_units='km', y_units='mm/hr', label='Ground', wavelength_ticks=wavelength_ticks, color='y', lw=1.0, ax=ax)

R_obs_spectrum, fft_freq = pysteps.utils.rapsd(pred_gan_depad, np.fft, d=1.0, return_freq=True)
wavelength_ticks = [512,256,128,64,32,16,8,4,2]
pysteps.plt.plot_spectrum1d(fft_freq, R_obs_spectrum, x_units='km', y_units='mm/hr', label='GAN Prediction', wavelength_ticks=wavelength_ticks, color='m', lw=1.0, ax=ax)


R_obs_spectrum, fft_freq = pysteps.utils.rapsd(pred_unet_depad, np.fft, d=1.0, return_freq=True)
wavelength_ticks = [512,256,128,64,32,16,8,4,2]
pysteps.plt.plot_spectrum1d(fft_freq, R_obs_spectrum, x_units='km', y_units='mm/hr', label='UNet Prediction', wavelength_ticks=wavelength_ticks, color='g', lw=1.0, ax=ax)

plt.legend()


In [ ]:
# Plot the intensity error
output_intensity = []
pred_intensity = []

for data in iter(val_loader):
    input, output = data
    input_np, output_np, pred = make_pred(input, output, 'gan')

    for j in range(16):
        output_depad = depad(output_np[j,0,:,:])
        output_intensity.append(output_depad.sum() / 192 / 128)
    for j in range(16):
        pred_depad = depad(pred[j,0,:,:])
        pred_intensity.append(pred_depad.sum() / 192 / 128)

fig = plt.figure(figsize=(18, 6))

fig.add_subplot(121)
plt.plot(output_intensity)
plt.plot(pred_intensity)

fig.add_subplot(122)
err_intensity = np.abs(np.array(output_intensity) - np.array(pred_intensity))
plt.plot(err_intensity)
plt.title("Intensity error per frame")

print(np.sum(err_intensity))

In [ ]:
# Plot the threat score (Critical Success Index: CSI) FOR THE ENTIRE DATASET with respect to threshold

# Compute the CSI for each frame first, then average the CSI
def csi_for_one(output, prediction, frame):     #Compute CSI for each of a particular frame
    TP, TN, FP, FN = 0., 0., 0., 0.
    output_depad = depad(output[frame, 0, :, :])
    prediction_depad = depad(prediction[frame, 0, :, :])

    for i in range(180):
        for j in range(120):
            if output_depad[i, j] == -1 and prediction_depad[i, j]  == -1: TN += 1.0
            elif output_depad[i, j]  == 1 and prediction_depad[i, j]  == 1: TP += 1.0
            elif output_depad[i, j]  == -1 and prediction_depad[i, j]  == 1: FP += 1.0
            elif output_depad[i, j]  == 1 and prediction_depad[i, j]  == -1: FN += 1.0
    CSI = TP / (TP + FN + FP + 1e-30)   #1e-30 for avoiding no rainfall condition where TP FN FP are all 0
    return CSI


#Compute the averaged CSI for entire dataset for a particular threshold
def getCSI(threshold, val_loader, model):
    CSI_list = []

    for batch in iter(val_loader):   
        input, output = batch

        if model == 'unet':
            input_np, output_np, pred = make_pred(input, output, 'unet')
        else:
            input_np, output_np, pred = make_pred(input, output, 'gan')

        #Turns all the element in the batch to -1/1 for True and False
        output_np[output_np < threshold] = -1
        output_np[output_np > threshold] = 1
        pred[pred < threshold] = -1
        pred[pred > threshold] = 1

        for i in range(batch_size):
            csi_onebatch = csi_for_one(output_np, pred, frame = i)
            if csi_onebatch != 0:       #if its 0, basically there's no rainfall and should not be involved in calculation since TP FN FP all 0
                CSI_list.append(csi_onebatch)

    avgCSI = sum(CSI_list) / len(CSI_list)
    return avgCSI


csi_w_threshold = []
threshold = []
for i in range(1, 18, 2):     #2,4,...,18,20
    csi = getCSI(i, val_loader, 'gan')
    csi_w_threshold.append(csi)    #append csi for this threshold in
    threshold.append(i)      #append this threshold value in

csi_w_threshold_unet = []
threshold_unet = []
for i in range(1, 18, 2):     #2,4,...,18,20
    csi = getCSI(i, val_loader, 'unet')
    csi_w_threshold_unet.append(csi)    #append csi for this threshold in
    threshold_unet.append(i)      #append this threshold value in


fig = plt.figure(figsize=(12, 8))
plt.plot(threshold_unet, csi_w_threshold_unet, label = 'UNet')
plt.plot(threshold, csi_w_threshold, label = 'GAN')
plt.legend()
plt.xlabel('Threshold (mm/hr)')
plt.ylabel('CSI')
plt.title("CSI with respect to Threshold", fontsize = 20)